In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
import dtcv
from shapely import wkt
from shapely.geometry import Polygon, LineString, Point
import json 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# San Diego Downtown Homless Computer Vision Package
`sandiegodata.org-downtown_cv-1` Last Update: 2019-09-09T01:06:55

_Files and code for analyzing san diego downtown homelessness  data with computer vision_




## Developer notes

To refresh the list of urls run

    $  bin/update_s3.sh <s3-profile>


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [gcp](notebooks/rundf.ipynb#gcp)**. Ground control points
* ** [intersection_regions](notebooks/rundf.ipynb#intersection_regions)**. Polygon transformations for each the intersections of each map
* ** [intersections](notebooks/rundf.ipynb#intersections)**. List of intersections.

In [3]:
gcp = dtcv.gcp_df()
gcp.head()

,image_url,x,y,width,height,intersection
0,http://ds.civicknowledge.org/downtownsandiego....,248,848,29,58,Ketner_Broadway
1,http://ds.civicknowledge.org/downtownsandiego....,573,851,25,61,State_Broadway
2,http://ds.civicknowledge.org/downtownsandiego....,575,244,31,31,State_A
3,http://ds.civicknowledge.org/downtownsandiego....,251,240,22,34,Ketner_A
4,http://ds.civicknowledge.org/downtownsandiego....,807,1226,41,41,Ketner_Broadway


In [21]:
def uninvert_point(p):
    x,y = p

    y = -y + 2000
    return (x,y)

def uninvert_poly(poly):

    return Polygon( uninvert_point(p) for p in poly_to_array(poly))

def poly_to_array(poly):

    try:
        
        return  np.array(poly.exterior.coords)[:-1]
    except AttributeError:
        return poly
    
df = dtcv.gcp_transform_df().rename(columns={'source':'source_inv'})

df['source'] = df.source_inv.apply(uninvert_poly)
df['source_area'] = df.source.apply(lambda v: v.area)

def bound_dims(v):
    (minx, miny, maxx, maxy) = v.bounds
    return round(maxx-minx,-1), round(maxy-miny,-1)

df['source_shape'] = df.source.apply(bound_dims)
df['source_shape_x'] = df.source_shape.apply(lambda v: v[0])
df['source_shape_y'] = df.source_shape.apply(lambda v: v[1])

from pathlib import Path 

def agg_inter(g):
    return ','.join(sorted(g.unique()))

t =gcp.rename(columns={'image_url':'url'}).groupby('url').agg({'intersection':agg_inter}).reset_index()\
    .rename(columns={'intersection':'intersections_id'})
df = df.merge(t, on='url')

df['year'] = df.url.apply(lambda v: int(Path(v).stem[:4]))
df['month'] = df.url.apply(lambda v: int(Path(v).stem[4:6]))


# some extra variant information
def map_names(r):

    ym = r.year*100 + r.month
    
    if  ym < 201600:
        return r.neighborhood+'_14'
    elif 201600 <= ym < 201704:
        return r.neighborhood+'_16'
    elif 201704 <= ym:
        return r.neighborhood+'_17'
    else:
        # Should not get here. 
        print(ym, r.year, r.month)
        return r.neighborhood+'_18'
        

df['map_name'] =  df.apply(map_names, axis=1)
   
# Different maps variants, based on the intersections
intersection_group_map = {
    'Ketner_A,Ketner_Broadway,State_A,State_Broadway':   'a', #'columbia',
    '11th_B,11th_Broadway,Front_B,Front_Broadway':       'a', #'core_columbia',
    '1st_Ash,1st_Cedar,9th_Ash,9th_Cedar':               'a', #'cortez',
    '16th_E,16th_Imperial,7th_E,7th_Imperial':           'a', #'east_village_a',
    '16th_Imperial,16th_Market,7th_Imperial,7th_Market': 'b', #'east_village_b',
    '16th_E,16th_Market,7th_E,7th_Market':               'c', #'east_village_c',
    '4th_Broadway,4th_K,6th_Broadway,6th_K':             'a', #'gaslamp',
    '3rd_G,3rd_K,Ketner_G,Ketner_Market':                'a'} #'marina'}

df['intersection_group'] = t.intersections_id.apply(lambda v: intersection_group_map[v] )


intersection_regions = df[['url', 'neighborhood', 'year', 'month','intersections_id',  'intersection_group', 'map_name',
         'source_inv', 'source', 'source_area', 'source_shape', 'source_shape_x', 'source_shape_y',
         'dest', 'matrix' ]]

intersection_regions.head().T


,0,1,2,3,4
url,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....,http://ds.civicknowledge.org/downtownsandiego....
neighborhood,columbia,columbia,columbia,columbia,columbia
year,2015,2016,2016,2016,2016
month,9,1,2,3,4
intersections_id,"Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway","Ketner_A,Ketner_Broadway,State_A,State_Broadway"
intersection_group,a,a,a,a,a
map_name,columbia_14,columbia_16,columbia_16,columbia_16,columbia_16
source_inv,"POLYGON ((262 1743, 262.5 1123, 585.5 1118.5, ...","POLYGON ((832.5 1161, 827.5 753.5, 1541.5 750....","POLYGON ((605 1426.5, 602 1086, 1188 1086, 118...","POLYGON ((844.5 1227.5, 848.5 777.5, 1626.5 77...","POLYGON ((627 1423, 626 1097.5, 1186.5 1095.5,..."
source,"POLYGON ((262 257, 262.5 877, 585.5 881.5, 590...","POLYGON ((832.5 839, 827.5 1246.5, 1541.5 1249...","POLYGON ((605 573.5, 602 914, 1188 914, 1185 5...","POLYGON ((844.5 772.5, 848.5 1222.5, 1626.5 12...","POLYGON ((627 577, 626 902.5, 1186.5 904.5, 11..."
source_area,202299,291446,197346,352782,182724


In [19]:
intersections = dtcv.intersections_df()
intersections.head().T

,0,1,2,3,4
geometry,POINT (6279769 1841430.99999986),POINT (6279780 1842558.99999987),POINT (6278946 1841436.99999983),POINT (6278953.17300012 1842563.14399985),POINT (6280315 1841426.99999984)
neighborhood,columbia,columbia,columbia,columbia,core_columbia
intersection,State_Broadway,State_A,Ketner_Broadway,Ketner_A,Front_Broadway


In [6]:
intersection_regions.columns

Index(['url', 'neighborhood', 'year', 'month', 'intersections_id',
       'intersection_group', 'map_name', 'source_inv', 'source', 'source_area',
       'source_shape', 'source_shape_x', 'source_shape_y', 'dest', 'matrix'],
      dtype='object')